# Decorators

Decorators are functions which modify the functionality of another function. They help to make our code more Pythonic.

### Everything in Python is an object

In [1]:
def hi(name = "Won"):
    print("Hi, " + name)

hi()

Hi, Won


In [2]:
# We can even assign a function to a variable like
greet = hi
greet()
# Don't use parentheses here because we are not calling the function hi
# instead we are just putting it into the greet variable.

Hi, Won


In [3]:
del hi
#hi() # NameError
greet()

Hi, Won


### Defining functions within functions
In Python we can define functions inside other functions

In [4]:
def hi(name = "Won"):
    print("Now you are inside the hi() function")
    
    def greet():
        print("Now you are in the greet() function")
    
    def welcome():
        print("Now you are in the welcome() function")
    
    greet()
    welcome()
    print("now you are back in the hi() function")
    
hi()

Now you are inside the hi() function
Now you are in the greet() function
Now you are in the welcome() function
now you are back in the hi() function


In [5]:
# welcome() # NameError: name 'welcome' is not defined

### Returning functions from within functions
It is not necessary to execute a function within another function, we can return it as an output as well

In [6]:
def hi(name = "Won"):
    def greet():
        print("Now you are in the greet() function")
    
    def welcome():
        print("Now you are in the welcome() function")
        
    if name == "Won":
        return greet
    
    else:
        return welcom
    
a = hi()
a 

<function __main__.hi.<locals>.greet>

The output above clearly shows that **'a'** now points to the greet() function in hi()

In [7]:
a()

Now you are in the greet() function


In the if/else clause we are returning greet and welcome, not greet() and welcome(). Its' because when you put a pair of parentheses after it, the function gets executed.

When we write a = hi(), hi() ets executed and because the name is Won by default, the function greet is returned. If we change the statement to a = hi(name = "David") the the welcome function will be returned.

### Giving a function as an argument to another function

In [8]:
def hi():
    return "Hi, Won!"

def doSomethingBeforeHi(func):
    print("I'm doing something before executing hi()")
    print(func())
    
doSomethingBeforeHi(hi)

I'm doing something before executing hi()
Hi, Won!


### Writing your first decorator

In [9]:
def new_decorator(func):
    def wrapFunc():
        print("before executing func()")
        func()
        print("after executing func()")
        
    return wrapFunc

def funcToDecorate():
    print("smells so bad")
    
funcToDecorate()

smells so bad


In [10]:
funcToDecorate = new_decorator(funcToDecorate)
funcToDecorate()

before executing func()
smells so bad
after executing func()


This is exactly what the decorators do in Python. They warp a function and modify its behaviour in one way or the another.

We can use the @ to make up a decorated function in a short way.

In [11]:
@new_decorator
def funcToDecorate():
    print("smells so bad")
    
funcToDecorate()

before executing func()
smells so bad
after executing func()


That is, ***@new_decorator*** is just a short way of saying ***funcToDecorate = new_decorator(funcToDecorate)***

Now there is one problem with our code.

In [12]:
print(funcToDecorate.__name__)

wrapFunc


Although we want to get its name "funcToDecorate", it was replaced by 'wrapFunc'. It overrode the name and docstirng of our function. Luckily Python provides us a simple function to sovle this problem and that is *functools.wraps*.

In [13]:
from functools import wraps

def new_decorator(func):
    @wraps(func)
    def wrapFunc():
        print("before executing func()")
        func()
        print("after executing func()")
        
    return wrapFunc

@new_decorator
def funcToDecorate():
    print("smells so bad!")
    
print(funcToDecorate.__name__)

funcToDecorate


## Use-cases
### Authorization
Decorators can help to check whether someone is authorized to use an endpoint in a web application. They are extensively used in Flask and Django.

In [14]:
from functools import wraps

def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.autorization
        if not auth or not chech_auth(auth.username, auth.password):
            authenticate()  
        return f(*args, **kwargs)
    return decorated

### Logging
Logging is another area where the decorators are useful.

In [15]:
from functools import wraps

def logit(f):
    @wraps(f)
    def with_logging(*args, **kwargs):
        print(f.__name__ + " was called")
        return f(*args, **kwargs)
    return with_logging

@logit
def addition_func(x):
    return x+x

result = addition_func(4)

addition_func was called


## Decorators with Arguments
@wraps is also a decorator. But, it takes an argument like any normal functions do. This is because when you use the @decorator syntax, you are applying a wrapper function with a single function as a parameter.

### Nesting a Decorator Within a Function

In [16]:
from functools import wraps

def logit(logfile = "out.log"):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_func(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            
            # open the logfile and append
            with open(logfile, 'a') as opened_file:
                # now we log to the specified logfile
                opened_file.write(log_string + '\n')
                
        return wrapped_func
    return logging_decorator

In [17]:
@logit()
def myfunc1():
    pass

myfunc1()
# A file called out.log now exists in my working direcotry, 
# with the above string

myfunc1 was called


In [18]:
@logit(logfile = "func2.log")
def myfunc2():
    pass

myfunc2()
# A file called func2.log now exists in my working directory,
# with the above string

myfunc2 was called


### Decorator Classes
Classes can also be used to build decorators. Let's rebuild logit as a class instead of a function.

In [19]:
class Logit(object):
    def __init__(self, logfile = "out.log"):
        self.logfile = logfile
        
    def __call__(self, func):
        log_string = func.__name + " was called"
        print(log_string)
        # open the logfile and append
        with open(logfile, "a") as opened_file:
            # now we log to the specified logfile
            opened_file.write(log_string)
        # now send a notification
        self.notify()
        
    def notify(self):
        # logit only logs, no more
        pass

This implementation has an additional advantage of being much cleaner than the nested function approach, and wrapping a function still will use the same syntax as before.

Now, let's subclass logit to add email functionality.

In [20]:
class EmailLogit(Logit):
    def __init__(self, email = "admin@snu.ac.kr", *args, **kwargs):
        self.email = email
        super(EmailLogit, self).__init__(*args, **kwargs)
        
    def notify(self):
        # sending an email to self.email, but not be inplemented here
        pass